In [2]:
# Imports
import warnings
import os
import sys

import numpy as np
import numpy.ma as ma
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import geopandas as gpd
from geopandas import GeoDataFrame as gdf
from geopandas import GeoSeries as gs
from shapely.geometry import Point, Polygon

import contextily as ctx
import earthpy as et
import earthpy.plot as ep
from earthpy import clip as cl

In [3]:
# Check path and set working directory.
wd_path = os.path.join(et.io.HOME, 'earth-analytics', 'data')
if os.path.exists(wd_path):
    os.chdir(wd_path)
else:
    print("Path does not exist")

In [3]:
# Download Data stored on figshare
# # Wdpa polygon files
# et.data.get_data(url="https://ndownloader.figshare.com/files/22847264")

# Free flowing rivers dataset
et.data.get_data(url="https://ndownloader.figshare.com/files/22936541")

# Future dams
et.data.get_data(url="https://ndownloader.figshare.com/files/22486157")

# Ramsar Sites
et.data.get_data(url="https://ndownloader.figshare.com/files/22507082")

'/home/jovyan/earth-analytics/data/earthpy-downloads/ramsar-site-data'

In [5]:
# Define the path to files
wdpa_polys_path = os.path.join(wd_path,
    "earthpy-downloads", "WDPA_Apr2020-shapefile-polygons", "WDPA_Apr2020-shapefile-polygons.shp")

ffr_0to5_path = os.path.join(wd_path,
    "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_0to5.shp")

ffr_5to10_path = os.path.join(wd_path,
    "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_5to10.shp")

ffr_10to20_path = os.path.join(wd_path,
    "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_10to20.shp")

ffr_20plus_path = os.path.join(wd_path,
    "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_20plus.shp")

country_borders_path = os.path.join(wd_path, "earthpy-downloads", "country-borders",
                                    "99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp")

ramsar_all = gpd.read_file(os.path.join(wd_path,
    "earthpy-downloads", "ramsar-site-data", "ramsar-boundaries",
    "features_publishedPolygon.shp"))

# Open the dams csv files with pandas
dams_source_path = os.path.join("earthpy-downloads", "future_dams_2015.csv")
dams_df = pd.read_csv(dams_source_path)

# Covert the pandas dataframe to a shapefile for plotting and set output path
dams_path = os.path.join('fhred-proposed-dams')
if not os.path.exists(dams_path):
    os.mkdir(dams_path)

In [ ]:
# Open the shapefiles with geopandas
wdpa_polys = gpd.read_file(wdpa_polys_path)
ffr_0to5 = gpd.read_file(ffr_0to5_path)
ffr_5to10 = gpd.read_file(ffr_5to10_path)
ffr_10to20 = gpd.read_file(ffr_10to20_path)
ffr_20plus = gpd.read_file(ffr_20plus_path)

In [ ]:
# Project wdpa polys
wdpa_polys = wdpa_polys.to_crs('epsg:4087')

# Project ffr data
ffr_0to5 = ffr_0to5.to_crs('epsg:4087')
ffr_5to10 = ffr_5to10.to_crs('epsg:4087')
ffr_10to20 = ffr_10to20.to_crs('epsg:4087')
ffr_20plus = ffr_20plus.to_crs('epsg:4087')

note about crs
World Equidistant Cylindrical, datum WGS84, units meters, EPSG 4087


In [ ]:
# Define the geometry for the points
geometry = [Point(xy) for xy in zip(dams_df.Lon_Cleaned, dams_df.LAT_cleaned)]
crs = {'init': 'epsg:4087'}
geo_df = gdf(dams_df, crs=crs, geometry=geometry)
geo_df.to_file(driver='ESRI Shapefile', filename=os.path.join(
    dams_path, 'proposed_dams.shp'))

In [ ]:
# Open the proposed dams shapefile with geopandas
proposed_dams = gpd.read_file(os.path.join(dams_path, "proposed_dams.shp"))

In [ ]:
# 50 km buffer
proposed_dams['geometry'] = proposed_dams.buffer(50 * 1000)

# overlay to cut to only dams and wdpa sites that overlap eachother
data_intersect_50 = gpd.overlay(wdpa_polys, proposed_dams, how='intersection')


# # Clip the proposed dams to the wdpa polys
# proposed_dams_cl = cl.clip_shp(proposed_dams, wdpa_polys)

# wdpa_buffer_50 = ((data_intersect_50.area.sum())/1000000).round(1)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 25))
country_borders.plot(ax=ax, color="lightyellow",
                     edgecolor="black", linewidth=2)
proposed_dams.plot(ax=ax,
                   markersize=15,
                   color='red', legend=True)